In [ ]:
!pip install openai==0.28
!pip install numpy
!pip install tiktoken
!pip install Gradio
!pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [ ]:
import openai
import csv
import json
import os
import numpy as np
from collections import defaultdict
import tiktoken
import gradio as gr

In [ ]:
openai.api_key = "sk-proj-HIa5j0kLt75BcRWyjYY9T3BlbkFJmoymtquLidzF5D4GW0TS"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Load CSV data in
csv_file_path = '/content/drive/MyDrive/Colab Notebooks/Lumen/DATASETLumen.csv'
cleaned_data = []

with open(csv_file_path, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        for cell in row:
            try:
                # Replace square brackets and inner double quotes that are problematic
                cell = cell.replace('["', '').replace('"]', '').replace('\\"', '"')

                # Load each cell as a JSON object
                cell_json = json.loads(cell)

                # Now that the content is clean, append to cleaned_data list
                cleaned_data.append(cell_json)
            except json.JSONDecodeError as e:
                print(f"JSON decode error for cell '{cell}': {e}")

jsonl_file_path = '/content/drive/MyDrive/Colab Notebooks/Lumen/DATASETLumen-json.jsonl'
# Write cleaned data to a JSONL file
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for item in cleaned_data:
        jsonl_file.write(json.dumps(item) + '\n')

JSON decode error for cell '{"messages": [{"role": "system", "content": "You are an overly friendly hospitality chatbot named Chatner who just loves to help people, and you're not satisfied unless the customer is completely satisfied."}, {"role": "user", "content": "Who are you?"}, {"role": "assistant", "content": "I'm Lumen, a Therapeutic AI Assitant designed to assist you. Tell me about yourself.
    "}]}': Invalid control character at: line 1 column 374 (char 373)
JSON decode error for cell '': Expecting value: line 1 column 1 (char 0)
JSON decode error for cell '': Expecting value: line 1 column 1 (char 0)
JSON decode error for cell '': Expecting value: line 1 column 1 (char 0)
JSON decode error for cell '': Expecting value: line 1 column 1 (char 0)
JSON decode error for cell '': Expecting value: line 1 column 1 (char 0)
JSON decode error for cell '': Expecting value: line 1 column 1 (char 0)
JSON decode error for cell '': Expecting value: line 1 column 1 (char 0)
JSON decode error

In [ ]:
#from OpenAI website to format data;  https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

# Next, we specify the data path and open the JSONL file

data_path = '/content/drive/MyDrive/Colab Notebooks/Lumen/DATASETLumen-json.jsonl'

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# We can inspect the data quickly by checking the number of examples and the first item

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Now that we have a sense of the data, we need to go through all the different examples and check to make sure the formatting is correct and matches the Chat completions message structure

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# Beyond the structure of the message, we also need to ensure that the length does not exceed the 4096 token limit.

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")

# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")


cost_per_100k_tokens = 0.80  # Cost for every 100,000 tokens
estimated_cost = ((n_epochs * n_billing_tokens_in_dataset) / 100000) * cost_per_100k_tokens
print(f"Estimated cost for fine-tuning: approximately ${estimated_cost:.2f}") #for actual cost based on current pricing

Num examples: 45
First example:
{'role': 'system', 'content': 'You are an empathetic chatbot named Lumen who provides mental health support to its users.'}
{'role': 'user', 'content': 'nothing much'}
{'role': 'assistant', 'content': 'Oh I see. Do you want to talk about something?'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 48, 530
mean / median: 134.9777777777778, 84.0
p5 / p95: 55.4, 363.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 9, 486
mean / median: 91.91111111111111, 39.0
p5 / p95: 12.8, 320.6

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning
Dataset has ~6074 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~18222 to

In [ ]:
# Function to save the dataset as a JSONL file
def save_to_jsonl(conversations, file_path):
    with open(file_path, 'w') as file:
        for conversation in conversations:
            json_line = json.dumps(conversation)
            file.write(json_line + '\n')

# Specify the path where you want to save the JSONL file in your Google Drive
jsonl_file_path = '/content/drive/MyDrive/Colab Notebooks/Lumen/DATASETLumen-json-clean.jsonl'
# Save the dataset to the specified file path
save_to_jsonl(dataset, jsonl_file_path)

In [ ]:
#Upload data for training
training_file_name = '/content/drive/MyDrive/Colab Notebooks/Lumen/DATASETLumen-json-clean.jsonl'

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

#Gives training file id
print("Training file id:", training_file_id)

Training file id: file-PzqG04CszWiU5jQkCtNhvDQ6


In [ ]:
#Create Fine-Tuning Job
suffix_name = "Lumen"

response = openai.FineTuningJob.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)

job_id = response["id"]

print(response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-oTw1VkHp18PYsI9psOn1trKX",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1722187561,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-4cYi4FGVVrXa9JLwwu8HiwRr",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-PzqG04CszWiU5jQkCtNhvDQ6",
  "hyperparameters": {
    "n_epochs": "auto",
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": "Lumen",
  "seed": 1754353780,
  "estimated_finish": null,
  "integrations": []
}


In [ ]:
#list events as fine-tuning progresses
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Step 86/135: training loss=2.02
Step 87/135: training loss=1.72
Step 88/135: training loss=1.61
Step 89/135: training loss=1.07
Step 90/135: training loss=1.44
Step 91/135: training loss=0.71
Step 92/135: training loss=0.84
Step 93/135: training loss=0.57
Step 94/135: training loss=1.26
Step 95/135: training loss=0.89
Step 96/135: training loss=0.96
Step 97/135: training loss=1.49
Step 98/135: training loss=1.51
Step 99/135: training loss=0.94
Step 100/135: training loss=1.86
Step 101/135: training loss=1.35
Step 102/135: training loss=1.27
Step 103/135: training loss=1.48
Step 104/135: training loss=1.67
Step 105/135: training loss=1.26
Step 106/135: training loss=0.35
Step 107/135: training loss=1.30
Step 108/135: training loss=0.35
Step 109/135: training loss=1.58
Step 110/135: training loss=0.50
Step 111/135: training loss=1.99
Step 112/135: training loss=0.47
Step 113/135: training loss=0.63
Step 114/135: training loss=1.56
Step 115/135: training loss=1.87
Step 116/135: training l

In [ ]:
#retrieve fine-tune model id
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)

{
  "object": "fine_tuning.job",
  "id": "ftjob-oTw1VkHp18PYsI9psOn1trKX",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1722187561,
  "finished_at": 1722187997,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0125:personal:lumen:9q23jzKV",
  "organization_id": "org-4cYi4FGVVrXa9JLwwu8HiwRr",
  "result_files": [
    "file-x3IdlJvUubDzEwOURWOFTqsX"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-PzqG04CszWiU5jQkCtNhvDQ6",
  "hyperparameters": {
    "n_epochs": 3,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": 17952,
  "error": {},
  "user_provided_suffix": "Lumen",
  "seed": 1754353780,
  "estimated_finish": null,
  "integrations": []
}

Fine-tuned model id: ft:gpt-3.5-turbo-0125:personal:lumen:9q23jzKV


In [ ]:
#Test it out
test_messages = []

system_message = "You are an empathetic chatbot named Lumen who provides mental health support to its users."
test_messages.append({"role": "system", "content": system_message})
user_message = "I am so stressed out"
test_messages.append({"role": "user", "content": user_message})

print(test_messages)

[{'role': 'system', 'content': 'You are an empathetic chatbot named Lumen who provides mental health support to its users.'}, {'role': 'user', 'content': 'I am so stressed out'}]


In [ ]:
#OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, #can test it against gpt-3.5-turbo to see difference
    messages=test_messages,
    temperature=0,
    max_tokens=500
)
print(response["choices"][0]["message"]["content"])

I'm sorry to hear that. What's been going on?


In [ ]:
#Gradio for a better UI

import gradio as gr
def generate_completion(user_prompt):
    hidden_context = ""
    messages = [
        {"role": "system", "content": hidden_context},
        {"role": "user", "content": user_prompt}
    ]
    response = openai.ChatCompletion.create(
        model=fine_tuned_model_id,
        messages=messages,
        max_tokens=100,
        temperature=0
    )
    return response['choices'][0]['message']['content'].strip()

iface = gr.Interface(fn=generate_completion,
                     inputs=gr.Textbox(lines=5, placeholder='Hi, how do you feel today?'),
                     outputs='text',
                     title="Lumen-The Mental Health Support Bot",
                     input_labels="Question",
                     output_labels="Response")

iface.launch(share=True, debug=True)

TypeError: BlockContext.__init__() got an unexpected keyword argument 'input_labels'